In [ ]:
from google.colab import drive
import os
import sys
drive.mount('/content/drive')
os.chdir("./drive/MyDrive/Project/")


Mounted at /content/drive


In [1]:
%%capture
!pip install np_utils
!pip install transformers
!pip install pickle5

In [ ]:
import numpy as np
import pandas as pd
import librosa
import pickle
import re
import pickle5 as pickle

from sklearn.model_selection import train_test_split,KFold,StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score,f1_score 
from sklearn.preprocessing import LabelEncoder,normalize,OneHotEncoder,LabelBinarizer


import keras
from keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.layers import  MaxPool2D,Dense,Input,Convolution2D,Flatten,Activation,Dropout
from keras import models,optimizers,losses
from keras.utils import np_utils
from keras.activations import softmax, relu 
from keras import backend as K

from transformers import RobertaTokenizer, RobertaModel, TFRobertaForSequenceClassification

import tensorflow as tf
import tensorflow_datasets as tfds



In [ ]:
%%capture
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')


In [ ]:
def convert_mfcc(path, bands, sample,duration):
    mf = np.empty(shape=(len(path), bands, 376, 1))
    total_len = sample*duration
    for i,one_path in enumerate(path):
        data,sr = librosa.load(one_path,sr=sample,res_type="kaiser_fast",duration=duration,offset=0.8) # offeset is the start
        data_len = len(data)
        if data_len > total_len:
            offset = np.random.randint((data_len - total_len))
            data = data[offset:(total_len+offset)]
        elif total_len > data_len:
            offset = np.random.randint(( total_len-data_len))
            data = np.pad(data, (offset, int(total_len) - len(data) - offset), "constant")
        else:
            data = np.pad(data, (0, int(total_len) - len(data)), "constant")
        mf[i,] = np.expand_dims(librosa.feature.mfcc(data, sr=sample, n_mfcc=bands), axis=-1)
    return mf
def get_model(bands):
    input = Input(shape=(bands,376,1))
    X = Convolution2D(64, (10,30), padding="same")(input)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    X = MaxPool2D()(X)
    X = Dropout(rate=0.2)(X)
    
    X = Convolution2D(64, (10,30), padding="same")(X)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    X = MaxPool2D()(X)
    X = Dropout(rate=0.2)(X)

    
    X = Convolution2D(64, (10,30), padding="same")(X)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    X = MaxPool2D()(X)
    X = Dropout(rate=0.75)(X)

    X = Flatten()(X)
    X = Dense(64)(X)
    X = Dropout(rate=0.2)(X)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    X = Dropout(rate=0.2)(X)
    
    out = Dense(7, activation=softmax)(X)
    model = models.Model(inputs=input, outputs=out)
    model.compile(optimizer=optimizers.Adam(0.001), loss=losses.categorical_crossentropy, metrics=['acc',f1_m])
    return model


In [ ]:

wav_list = []
folders = ["out"]
empty = []
for i in folders:
    da= os.listdir(f"MELD/{i}")    
    for k in da:
        one_path = f"MELD/{i}/"+ k
        if  '.wav' in k: # '.mp3' in k or
            if os.path.getsize(one_path)!=0:
                wav_list.append(one_path) 
                # durations.append(eyed3.load(one_path).info.time_secs)
            else:
                empty.append(one_path) 
                



In [ ]:
train = pd.read_csv("train_sent_emo.csv")
test = pd.read_csv("test_sent_emo.csv")
validation = pd.read_csv("dev_sent_emo.csv")


In [ ]:
label = []
index = []
for i in wav_list:
    # print(i)
    if '(1)' not in i:
        dia = int(re.split('/|_|\.|test', i)[-3][3:])
        utt = int(re.split('/|_|\.|test', i)[-2][3:])
        e = train[(train["Dialogue_ID"]==dia) & (train["Utterance_ID"]==utt)]['Emotion'].tolist()[0]
        index.append(train[(train["Dialogue_ID"]==dia) & (train["Utterance_ID"]==utt)]['Emotion'].index.tolist()[0])


LOAD precomputed mfcc bands

In [ ]:
mfcc = []
with open('mfcc_meld_pc_4.pkl', 'rb') as handle:
    mfcc = pickle.load(handle)
label = []
with open('label_meld_pc_4.pkl', 'rb') as handle:
    label = pickle.load(handle)
index = []
with open('info_3.pkl', 'rb') as handle:
    index = pickle.load(handle)

In [ ]:
train = pd.read_csv("train_sent_emo.csv")
test = pd.read_csv("test_sent_emo.csv")
validation = pd.read_csv("dev_sent_emo.csv")
train = train.loc[index, :]
s = pd.Series(list(range(len(train))))
train=train.set_index(s)


In [ ]:
sample=48000
duration=4
bands = 30

teee = range(len(mfcc))

X_train, X_test, train_dat, test_dat = train_test_split(mfcc
                                                    , teee
                                                    , test_size=0.2
                                                    , shuffle=True
                                                    , random_state=42)

y_train = [label[i] for i in train_dat]
y_test = [label[i] for i in test_dat]

rrrr = train.loc[test_dat, :]
train  = train.loc[train_dat, :]
train = train.append(test, ignore_index=True)
train = train.append(validation, ignore_index=True)
test = rrrr


In [ ]:
from sklearn.utils import class_weight
class_weights = dict(zip(range(len(np.unique(y_train))), class_weight.compute_class_weight('balanced', np.unique(y_train), 
                y_train))) 

In [ ]:
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

In [ ]:
# from https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# (64, (10,30)
def get_model(bands):
    input = Input(shape=(bands,376,1))
    X = Convolution2D(16, (5,5), padding="same")(input)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    X = MaxPool2D()(X)
    X = Dropout(rate=0.2)(X)
    
    X = Convolution2D(16, (5,5), padding="same")(X)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    X = MaxPool2D()(X)
    X = Dropout(rate=0.2)(X)
    X = Convolution2D(16, (5,5), padding="same")(X)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    X = MaxPool2D()(X)
    X = Dropout(rate=0.2)(X)

    
    X = Convolution2D(16, (5,5), padding="same")(X)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    X = MaxPool2D()(X)
    X = Dropout(rate=0.7)(X)

    X = Flatten()(X)
    X = Dense(64)(X)
    X = Dropout(rate=0.2)(X)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    X = Dropout(rate=0.2)(X)
    
    out = Dense(7, activation=softmax)(X)
    model = models.Model(inputs=input, outputs=out)
    model.compile(optimizer=optimizers.Adam(0.001), loss=losses.categorical_crossentropy, metrics=['acc',f1_m])
    return model


In [ ]:

mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std
checkpoint = ModelCheckpoint(filepath='mel.h5', 
                             monitor='val_f1_m',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')
# Build CNN model 
model = get_model(bands)

model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), 
                    batch_size=64, verbose = 2, epochs=200 ,callbacks=[checkpoint]) #, class_weight=class_weights

In [ ]:
# %%capture cap --no-stderr
# KF = KFold(n_splits=5, random_state=42, shuffle=True) #Stratified
# for train_index, test_index in KF.split(mfcc,label):
#     checkpoint = ModelCheckpoint(filepath='e.h5', 
#                              monitor='val_f1_m',
#                              verbose=1, 
#                              save_best_only=True,
#                              mode='max')
#     X_train, X_test = mfcc[train_index], mfcc[test_index]
#     y_train, y_test = np.asarray(label)[train_index], np.asarray(label)[test_index]

#     lb = LabelEncoder()
#     y_train = np_utils.to_categorical(lb.fit_transform(y_train))
#     y_test = np_utils.to_categorical(lb.fit_transform(y_test))

#     mean = np.mean(X_train, axis=0)
#     std = np.std(X_train, axis=0)

#     X_train = (X_train - mean)/std
#     X_test = (X_test - mean)/std

#     model = get_model(bands)
#     model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), 
#                         batch_size=64, verbose = 2, epochs=200,callbacks=[checkpoint]) 
    
    


In [ ]:

with open('output_meld.txt', 'w') as f:
    f.write(cap.stdout)

In [ ]:
model.load_weights('mel.h5')

In [ ]:
backup =  model.predict(X_test)

In [ ]:
accuracy_score(y_test.argmax(axis=1),backup.argmax(axis=1))
# accuracy_score(g,backup.argmax(axis=1))

In [ ]:
encoder = LabelBinarizer()
encoder.fit(train['Emotion'])
train_label  = encoder.transform(train['Emotion'])
test_label  = encoder.transform(test['Emotion'])
lab = LabelEncoder().fit(train['Emotion'])
f  = lab.transform(train['Emotion'])
g  = lab.transform(test['Emotion'])

In [ ]:
max_length = 128
batch_size = 64


def map_example_to_dict(input_ids, attention_masks, label):
    # print(label[0])
    return {
      "input_ids": input_ids,
      "attention_mask": attention_masks,
           }, label[0]
def encode_examples(ds, limit=-1):
  input_id = []
  attention_mask = []
  labels = []
  for one_data, label in tfds.as_numpy(ds):
    bert_input = tokenizer.encode_plus(one_data.decode(),add_special_tokens=True,max_length=max_length,pad_to_max_length=True,return_attention_mask=True,)
    input_id.append(bert_input['input_ids'])
    attention_mask.append(bert_input['attention_mask'])
    labels.append([label])

  return tf.data.Dataset.from_tensor_slices((input_id,
                                             attention_mask,
                              labels)).map(map_example_to_dict)

In [ ]:

training_sentences_modified = tf.data.Dataset.from_tensor_slices((train['Utterance'], train_label)) 
testing_sentences_modified = tf.data.Dataset.from_tensor_slices((test['Utterance'],test_label)) 

ds_train_encoded = encode_examples(training_sentences_modified).batch(batch_size) 
ds_test_encoded = encode_examples(testing_sentences_modified).batch(batch_size)

In [ ]:
learning_rate = 1.75e-5
number_of_epochs = 5

class ModelMetrics(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.count_n = 1
        self.best = 0

    def on_epoch_end(self, batch, logs={}):
        y_val_pred_train = tf.nn.softmax(self.model.predict(ds_train_encoded)['logits'])
        y_pred_argmax_train = tf.math.argmax(y_val_pred_train, axis=1)

        y_val_pred = tf.nn.softmax(self.model.predict(ds_test_encoded)['logits'])
        y_pred_argmax = tf.math.argmax(y_val_pred, axis=1)
        training_copy = {}
        training_copy['predicted'] = y_pred_argmax_train
        testing_copy = {}
        testing_copy['predicted'] = y_pred_argmax 

        f1_s = accuracy_score(g, testing_copy['predicted'].numpy())
        f2_s = accuracy_score(f, training_copy['predicted'].numpy())
        micro = f1_score(g, testing_copy['predicted'].numpy(), average='micro')
        weight = f1_score(g, testing_copy['predicted'].numpy(), average='weighted')
        print('\n accuracy score is :', f1_s, micro, weight)
        if self.best < f1_s:
            self.model.save_pretrained('rob.h5')
            self.best = f1_s
            print(f"saving model:{f1_s} better than {self.best}")
        else:
            print(f"{self.best} better accuracy not saving")
        self.count_n += 1

metrics = ModelMetrics()
# model initialization
model = TFRobertaForSequenceClassification.from_pretrained("roberta-base", num_labels = 7)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-05)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss) # , metrics=[metric]

histCallback = model.fit(ds_train_encoded, epochs=number_of_epochs,
          validation_data=ds_test_encoded, callbacks=[metrics]) # metrics is giving issues ,checkpoint
                                                           

In [ ]:
model = TFRobertaForSequenceClassification.from_pretrained("rob.h5", num_labels = 7)
# t2 = model.predict( ds_test_encoded  )
t2 = model.predict( ds_test_encoded  )
textual = tf.nn.softmax(model.predict(ds_test_encoded)['logits']).numpy()

In [ ]:
accuracy_score(g, textual.argmax(axis=1))

In [ ]:
textual = tf.nn.softmax(model.predict(ds_test_encoded)['logits']).numpy()


In [ ]:
best =0.5*backup+textual
accuracy_score(g, best.argmax(axis=1))

In [ ]:
from sklearn.metrics import f1_score
f1_score(g, best.argmax(axis=1), average='weighted')